# Introduction

In this exercise, you will create and submit predictions for a Kaggle competition. You can then improve your model (e.g. by adding features) to apply what you've learned and move up the leaderboard.

Begin by running the code cell below to set up code checking and the filepaths for the dataset.

In [1]:
# Set up code checking
# Set up filepaths
import os
os.chdir(os.path.join(os.path.expanduser('~'), 'kaggle'))

In [2]:
# Import helpful libraries
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

## Load data to pd.Df

In [3]:
iowa_file_path = 'data/house-price-data/train.csv'
home_data: pd.DataFrame = pd.read_csv(iowa_file_path)

test_data_path = 'data/house-price-data/test.csv'
test_data: pd.DataFrame = pd.read_csv(test_data_path)

In [4]:
# sort data into numeric and categoric variables
y = home_data['SalePrice']
home_data.drop(columns=['SalePrice'], inplace=True)

numeric_features = [col for col in home_data.columns if is_numeric_dtype(home_data[col].dtype)]
categorical_features = [col for col in home_data.columns if is_string_dtype(home_data[col].dtype)]
# categorical values which appear as numbers
hidden_categorical = ['MSSubClass']
categorical_features.extend(hidden_categorical)
numeric_features = [n for n in numeric_features if (n not in hidden_categorical)]
print("numerical features")
print(numeric_features)
print('categorical features')
print(categorical_features)

numerical features
['Id', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
categorical features
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageF

## Fill mising data, encode categorical features

In [5]:
# get cols with missing data
cols_with_na = [c for c in home_data.columns if home_data[c].isna().any()]

# print([1 if c else 0 for c in home_data[cols_with_na[1]].isna()])
# add indicator data was missing
for cna in cols_with_na:
    home_data[f'{cna}_was_missing'] = [1 if c else 0 for c in home_data[cols_with_na[1]].isna()]
    test_data[f'{cna}_was_missing'] = [1 if c else 0 for c in test_data[cols_with_na[1]].isna()]
    numeric_features.append(f'{cna}_was_missing')

for nf in numeric_features:
    home_data[nf].fillna(home_data[nf].mean(), inplace=True)
    test_data[nf].fillna(test_data[nf].mean(), inplace=True)

for cf in categorical_features:
    home_data[cf].fillna(home_data[cf].mode()[0], inplace=True)
    test_data[cf].fillna(test_data[cf].mode()[0], inplace=True)

# onehotencoding for categorical features
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

cat_features_df = pd.DataFrame(ohe.fit_transform(home_data[categorical_features]), columns=ohe.get_feature_names_out())

test_cat_features_df = pd.DataFrame(ohe.transform(test_data[categorical_features]), columns=ohe.get_feature_names_out())

home_data = pd.concat([home_data[numeric_features], cat_features_df], axis=1)
test_data = pd.concat([test_data[numeric_features], test_cat_features_df], axis=1)
print(test_data.shape, home_data.shape)

(1459, 322) (1460, 322)


### Feature selection

In [10]:
features = home_data.columns
print(features)
# Select columns corresponding to features, and preview the data
X = home_data[features]

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.4, random_state=1)

Index(['Id', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'MSSubClass_60', 'MSSubClass_70', 'MSSubClass_75', 'MSSubClass_80',
       'MSSubClass_85', 'MSSubClass_90', 'MSSubClass_120', 'MSSubClass_160',
       'MSSubClass_180', 'MSSubClass_190'],
      dtype='object', length=322)


## Select model

In [12]:
from sklearn.linear_model import LinearRegression, BayesianRidge 
from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_absolute_error

models = {'random forest': RandomForestRegressor(criterion='absolute_error'), 
        # 'linear regression': LinearRegression(copy_X=True), 
        # 'lgbm': LGBMRegressor(), 
        # 'xgregressor': XGBRegressor(), 
        'Gradient boost regressor': GradientBoostingRegressor(loss='absolute_error'),
        # 'SVR': SVR(),
        # 'bayes ridge': BayesianRidge(),
        }

def get_mae(model):
    model.fit(train_X, train_y)
    model_predictions = model.predict(val_X)
    return mean_absolute_error(model_predictions, val_y)

for model_name in models:
    model = models[model_name]
    mae = get_mae(model)
    print(f"Model: {model_name}; MAE: {mae: .0f}")
    print()
# Define a random forest model
# rf_model = RandomForestRegressor(random_state=1)
# rf_model.fit(train_X, train_y)
# rf_val_predictions = rf_model.predict(val_X)
# rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

# relative_error = rf_val_mae/y.mean()
# print(f"Validation MAE for Random Forest Model: {rf_val_mae: .0f}")
# print(f"Relative error: {relative_error}")

Model: random forest; MAE:  18330

Model: Gradient boost regressor; MAE:  17486



## Hyperparam tuning

In [28]:
# hyperparams for random forest

forest_random_grid = {'n_estimators': [int(x) for x in np.logspace(start=1, stop=2.4, num=10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(10, 80, num = 8)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf':[1, 2, 4],
               'bootstrap': [True, False]}

print(forest_random_grid)

{'n_estimators': [10, 14, 20, 29, 41, 59, 85, 122, 175, 251], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [33]:
# hyperparams for gradient boost regressor
# {'n_estimators':[200, 300, 400, 500, 600], 'max_depth':[2,3, 4, 5], 'subsample':[.5,.75,1], 'random_state':[1]}
GBR_random_grid = {'n_estimators': [int(x) for x in np.logspace(start=1, stop=2.4, num=10)],
                'max_depth': [1, 2, 3, 4, 5, 7, 10, 13, 18], 
                'subsample': [0.5, 0.75, 1], 
                'learning_rate': [1, 0.5, 0.25, 0.1, 0.05], 
                'min_samples_split': np.linspace(0.01, 0.5, 10, endpoint=True)}
print(GBR_random_grid)

{'n_estimators': [10, 14, 20, 29, 41, 59, 85, 122, 175, 251], 'max_depth': [1, 2, 3, 4, 5, 7, 10, 13, 18], 'subsample': [0.5, 0.75, 1], 'learning_rate': [1, 0.5, 0.25, 0.1, 0.05], 'min_samples_split': array([0.01      , 0.06444444, 0.11888889, 0.17333333, 0.22777778,
       0.28222222, 0.33666667, 0.39111111, 0.44555556, 0.5       ])}


In [34]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer
search_grids = {'random forest': forest_random_grid, 
                'Gradient boost regressor': GBR_random_grid}

for model_name in models:
    model_to_optimise = models[model_name]

    param_grid = search_grids[model_name]
    search = RandomizedSearchCV(estimator=model_to_optimise, 
                                param_distributions=param_grid, 
                                scoring=make_scorer(mean_absolute_error, greater_is_better=False), 
                                n_iter=20, 
                                n_jobs=-1, 
                                cv=3)

    search.fit(X, y)
    print(f'Model: {model_name}')
    print(search.best_params_)
    print(search.best_score_)

Model: random forest
{'n_estimators': 85, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}
-17570.286801368795
Model: Gradient boost regressor
{'subsample': 0.75, 'n_estimators': 85, 'min_samples_split': 0.22777777777777777, 'max_depth': 3, 'learning_rate': 0.1}
-17316.146014511974


In [ ]:
full_model = GradientBoostingRegressor(max_depth=3, n_estimators=85, subsample=0.75, loss='absolute_error')
full_model.fit(X, y)

Now, read the file of "test" data, and apply your model to make predictions.

In [ ]:
# path to file you will use for predictions
# test_data_path = '../input/test.csv'

# # read test data file using pandas
# test_data = pd.read_csv(test_data_path)

# # create test_X which comes from test_data but includes only the columns you used for prediction.
# # The list of columns is stored in a variable called features
test_X = test_data[features]

# # make predictions which we will submit. 
test_preds = full_model.predict(test_X)
print(test_preds)
print(type(test_preds), len(test_preds))

Before submitting, run a check to make sure your `test_preds` have the right format.

# Generate a submission

Run the code cell below to generate a CSV file with your predictions that you can use to submit to the competition.

In [ ]:
# Run the code to save predictions in the format used for competition scoring

output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('code/house-prices-competition/submission_grad_boost_1.csv', index=False)